In [1]:
import os
import pandas as pd
import glob
import pickle

In [2]:
class Preprocess:
    def __init__(self):
        super().__init__()
        self.file = ''
        self.keys,self.replace,self.dicts = self.load_keys_and_replacements()

    def load_keys_and_replacements(self):
        keys = list(map(lambda x:x.replace('\n',''), self.read_txt('keys/keywords.txt')))
        replace = list(map(lambda x:x.replace('\n',''), self.read_txt('keys/replace.txt')))
        dicts = {}
        for line in replace:
            x,y = line.split('==')[0].replace(' ',''), line.split('==')[1].replace(' ','')
            dicts[x] = y
        return keys,replace, dicts
        
    def read_txt(self,path):
        f = open(path)
        lines = f.readlines()
        f.close()
        return lines
    
    def tokenize(self,txt):
        return txt.split(' ')
    
    def replace_symbol(self,word):
        wordlist = []
        try:
            type(int(word))
            wordlist.append('INT')
        except:
            word = list(word)
            for i,l in enumerate(word):
                try:
                    repl = self.dicts[l]
                except:
                    repl = None
                    
                if repl != None:
                    l = " "+repl+' '
                wordlist.append(l)
        
        return " ".join("".join(wordlist).strip().split("  "))
    
    
    
    def Converter(self,query):
        query = query.lower()
        bb = list(map(self.replace_symbol, query.split(' ')))
        bb = ' '.join(bb)
        tokenized = ' '.join(list(map(self.replace_symbol,bb.split(' '))))
        split = tokenized.split(' ')
        for i,word in enumerate(tokenized.split(' ')):
            if word.upper() in self.keys:
                split[i] = word.upper()
        return " ".join(split)
    
        

In [3]:
class PrepareDataset(object):
    def __init__(self):
        
        self.preprocess = Preprocess()
        self.all_vocabs = []
    def open_file(self,filename,label,safe=False,limit=0):
        f = open(filename,'r')
        if limit == 0:
            txt = f.readlines()
        else:
            txt = f.readlines()[:limit]
        f.close()
        text = []
        for sent in txt:
            if len(sent) > 0:
                sent_list = []
                sent = sent.replace("\n",'').split(" ")
                for word in sent:
                    if len(word)> 0:
                        sent_list.append(word)
                if safe:
                    text.append(" ".join(sent_list))
                else:
                    text.append(self.preprocess.Converter(" ".join(sent_list)))
        dict = {'text':text,'label':[label for i in range(len(text))]}
        data = pd.DataFrame(dict)
        self.all_vocabs.append([sent for sent in data['text']])
        return data
    
    def save_multiple(self,paths_list):
        all_pds = []
        for file in paths_list:
            data = self.open_file(file,file)
            all_pds.append(data)
        return all_pds
    
            
        
    
    

In [17]:
class Vectorizer(object):
    def __init__(self ):
        self.vocab = set()
        self.labels = set()
        self.index_2_token = {}
        self.token_2_index = {}
        self.labels_2_index = {}
        self.index_2_labels = {}
        
    def build_vocab(self,csv_file):
        df = csv_file
        
        #lets build the vocab
        for data in df.iterrows():
            text, label = data[1]['text'], data[1]['label']
            self.add_token(text)
            self.labels.add(label)
    
            
    def tokenize(self,txt):
        return txt.split(' ')
    
    def add_token(self,txt):
        tokenized = self.tokenize(txt)
        for tok in tokenized:
            self.vocab.add(tok)
            
    def get_vocab(self):
        return list(self.vocab)
    
    def get_labels(self):
        return {lbl:i for i,lbl in self.labels}
    
        

In [19]:
class Build_Dataset():
    def __init_(self, pds):
        self.all_pds= pd.concat(pds)
        self.vectorizer = Vecotrizer()
        
        for pd in pds:
            self.vectorizer.build_vocab(pd)
            
        
    

In [18]:
Preprocess().Converter('<script>alert(1)</script')

'LT script GT alert LPRN INT RPRN LT SLSH script'

In [ ]:
Build_Dataset